<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>

## The Geo Dataframe

The geodata frame is an extended data frame where every row represents an spatial element (point, line, polygon, multipolygon).

Historically, the most common file type that stores spatial elements is the shapefile. Let's take a look at some of them:

1. Create a repository named: introgeodf.
2. Clone that repo to a local folder in your computer.
3. In that local folder in your computer, create a folder named **maps**.
4. Go to this [website](https://www.efrainmaps.es/english-version/free-downloads/world/).
5. Download three map files into the folder **maps** in your computer: countries, cities, and rivers.

You may see something like this:

<img title="a title" alt="Alt text" src="mapsFolderImage.png">

You can decompress those files:

<img title="a title" alt="Alt text" src="folderRar_1.png">

Now, take a look a **World_Countries**:

<img title="a title" alt="Alt text" src="imageCountries_shp.png">

There, you see that this **one map** requires **several files**. That is the nature of the shapefile. 

Let's read the file with the help of **geopandas**:

In [ ]:
import os, geopandas as gpd

countries=gpd.read_file(os.path.join("maps","World_Countries","World_Countries.shp"))

Let's use some familiar code:

In [ ]:
# what is it?
type(countries)

In [ ]:
# dimensions
countries.shape

In [ ]:
# names
countries.columns

In [ ]:
# some content
countries.head()

In [ ]:
# any missing values?
countries[countries.isna().any(axis=1)]

In [ ]:
# types
countries.info()

# Basic Plotting

As you see, every pandas command is working, but now we have a new column type: **geometry**.

In [ ]:
# plot simple

countries.plot()

Let's open the other maps:

In [ ]:
rivers=gpd.read_file(os.path.join("maps","World_Hydrography","World_Hydrography.shp"))
cities=gpd.read_file(os.path.join("maps","World_Cities","World_Cities.shp"))

In [ ]:
# visualizing rivers
rivers.plot()

In [ ]:
# visualizing cities
cities.plot()

## Some visual adjustments

In general you need to adjust:
* color of the polygon/line/point (see [here](https://matplotlib.org/stable/gallery/color/named_colors.html))
* thickness of lines,
* line style (see [here](https://matplotlib.org/stable/gallery/lines_bars_and_markers/linestyles.html))
* size of points, 
* point style (see [here](https://matplotlib.org/stable/api/markers_api.html))
* and the transparency level (you can find a list of colors ) . Let's see:

In [ ]:
countries.plot(facecolor="azure",#color of polygon fill
               edgecolor='black', #color of lines
               linewidth=0.1) #thickness of lines

In [ ]:
rivers.plot(edgecolor='blue', 
            linewidth=1,
            linestyle='dotted')

In [ ]:
cities.plot(marker='.', # marker type
            color='red', 
            markersize=1,
            alpha=0.3) # transparency

### Multiple layers of maps

This requires to plot a map **on top** of the other (order matters). 

It is a simple process, but the first step is verify that all have the same projection (**CRS**):

In [ ]:
countries.crs

In [ ]:
cities.crs

In [ ]:
rivers.crs

They all have the same crs, but if they did not, you have to use **to_crs()**:

In [ ]:
# changing crs in cities and rivers to be the same as countries

cities=cities.to_crs(countries.crs)
rivers=rivers.to_crs(countries.crs)

You can start by creating the layer on the back (the base), and add layers on top:

In [ ]:
base = countries.plot(facecolor="white", edgecolor='black', linewidth=0.1,figsize=(12,12))
cities.plot(marker='.', color='red', markersize=1,alpha=0.7,
            ax=base) # on top of..
rivers.plot(edgecolor='blue', linewidth=0.4,
            ax=base)# on top of..

A nice feature would be to represent the map in an interactive way:

In [ ]:
import folium


m = cities.explore(color="red", 
                   name="cities")#optional

m = rivers.explore(m=m, color="blue",
                   name="rivers")#optional
#folium.LayerControl().add_to(m) #optional
m

You can subset your map as you did in classic data frames:

In [ ]:
brasil=countries[countries.COUNTRY=='Brazil']

Then you can keep the **parts** of the other maps that intersect with the sub-polygon:

In [ ]:
cities_clipped = gpd.clip(gdf=cities,
                          mask=brasil)
rivers_clipped = gpd.clip(gdf=rivers,
                               mask=brasil)

Then, you can plot the clipped version:

In [ ]:
base = brasil.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
cities_clipped.plot(marker='+', color='red', markersize=15,ax=base)
rivers_clipped.plot(edgecolor='blue', linewidth=0.5,ax=base)

The interactive alternative for this last case could require to set the **folium** map to a particular coordinate. Let's finde the one for Brazil here: [https://www.geodatos.net/en/coordinates](https://www.geodatos.net/en/coordinates):

In [ ]:
brazilCoord=[-14.235004, -51.92528]

In [ ]:
m = cities.explore(location=brazilCoord,
                   zoom_start=4.5,
                   tiles='CartoDB positron',
                   color='red',
                   name="cities") #optional
m = rivers.explore(m=m, color="blue",
                   name="rivers")#optional
# folium.LayerControl().add_to(m) #optional
m

## Exporting maps

You may need a couple of things here:

- Save the map as an image.
- Save the files (not the images).

Let's see:

In [ ]:
# save the map as image
import matplotlib.pyplot as plt

base = brasil.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
map2=cities_clipped.plot(marker='+', color='red', markersize=15,ax=base)
mapEnd=rivers_clipped.plot(edgecolor='blue', linewidth=0.5,ax=base)
plt.savefig(os.path.join("maps",'mapEnd.jpg'))

We have not changed the initial map, but we could export the maps into a different file type (not shapefiles):

In [ ]:
countries.to_file(os.path.join("maps","worldMaps.gpkg"), layer='countries', driver="GPKG")
cities.to_file(os.path.join("maps","worldMaps.gpkg"), layer='cities', driver="GPKG")
rivers.to_file(os.path.join("maps","worldMaps.gpkg"), layer='rivers', driver="GPKG")

The most important thing, now that you have saved these several maps into one file is that, once in GitHub, you can call the map with the url link...


<img title="a title" alt="Alt text" src="getLinkGit.png">

Let me save the link recovered:

In [ ]:
worldMaps='https://github.com/CienciaDeDatosEspacial/code_and_data/raw/main/maps/worldMaps.gpkg'

You can ask what layers are present:

In [ ]:
from  fiona import listlayers

listlayers(worldMaps)

Now you are confident what to request:

In [ ]:
countries=gpd.read_file(worldMaps,layer='countries')
cities=gpd.read_file(worldMaps,layer='cities')
rivers=gpd.read_file(worldMaps,layer='rivers')

As you see, it works great:

In [ ]:
base = countries.plot(facecolor='gainsboro')
cities.plot(ax=base, markersize=0.5, color='red') 
rivers.plot(ax=base, linewidth=0.5)

Exercise:

1. Create a repo: **simpleplot**
2. Clone that repo to your computer.
3. Get **three** _SHP_ files from any **country** (do not use islands or small territories). You should have polygons, lines and points. 
4. Plot:
    4.1 Each one independently.
    4.2 All together 
    4.3 All together interactively
5. Save the NON interactive map as an image.
6. Save the three geodataframes into one geopackage file.
7. Get the link for the geopackage and make sure it is working well.